# 01 유저로부터 비디오 가져오기
###### home.pug
```pug

doctype html
html(lang="en")
  head
    meta(charset="UTF-8")
    meta(http-equiv="X-UA-Compatible", content="IE=edge")
    meta(name="viewport", content="width=device-width, initial-scale=1.0")
    title Joom
    //- MVP CSS 사용
    link(rel="stylesheet", href="https://unpkg.com/mvp.css@1.12/mvp.css") 
  body 
    header
      h1 Joom 
    main 
      div#myStream
        video#myFace(autoplay, playsinline, width='400', height='400')
        button#mute Mute
        button#camera Turn Camera Off
    script(src="/socket.io/socket.io.js")
    script(src="/public/js/app.js")

```


###### app.js

https://developer.mozilla.org/en-US/docs/Web/API/MediaDevices/getUserMedia

```javascript
const socket = io();

const myFace = document.getElementById("myFace");

let myStream;

async function getMedia() {
  try {
    myStream = await navigator.mediaDevices.getUserMedia({
      audio: true,
      video: true,
    });
    console.log(myStream);
  } catch (error) {
    console.log(error);
  }
}

getMedia();
```

---

# 02 마이크 음소거, 음소거 해제 버튼, 카메라 기능

### call Controlls


https://developer.mozilla.org/en-US/docs/Web/API/MediaDevices

###### app.js
```javascript
const socket = io();

const myFace = document.getElementById("myFace");
const muteBtn = document.getElementById("mute");
const cameraBtn = document.getElementById("camera");
const camerasSelect = document.getElementById("cameras");

let myStream;
let muted = false;
let cameraOff = false;

// https://developer.mozilla.org/ko/docs/Web/API/MediaDevices/enumerateDevices
async function getCameras() {
  try {
    const devices = await navigator.mediaDevices.enumerateDevices();
    const cameras = devices.filter((device) => {
      return device.kind === "videoinput";
    });

    cameras.forEach((camera) => {
      const option = document.createElement("option");
      option.value = camera.deviceId;
      option.innerText = camera.label;
      camerasSelect.appendChild(option);
    });

    console.log(cameras);
    // console.log(devices);
  } catch (error) {
    console.log(error);
  }
}

// https://developer.mozilla.org/en-US/docs/Web/API/MediaDevices/getUserMedia
async function getMedia() {
  try {
    myStream = await navigator.mediaDevices.getUserMedia({
      audio: true,
      video: true,
    });
    myFace.srcObject = myStream;
    //
    await getCameras();
  } catch (error) {
    console.log(error);
  }
}

getMedia();

function handleMuteClick() {
  myStream.getAudioTracks().forEach((track) => {
    track.enabled = !track.enabled;
    return;
  });
  if (!muted) {
    muteBtn.innerText = "UnMute";
    muted = true;
  } else {
    muteBtn.innerText = "Mute";
    muted = false;
  }
}
function handleCameraClick() {
  myStream
    .getVideoTracks()
    .forEach((track) => (track.enabled = !track.enabled));
  if (cameraOff) {
    cameraBtn.innerText = "Turn Camera Off";
    cameraOff = false;
  } else {
    cameraBtn.innerText = "Turn Camera On";
    cameraOff = true;
  }
}

muteBtn.addEventListener("click", handleMuteClick);
cameraBtn.addEventListener("click", handleCameraClick);

```

###### home.pug
```pug
doctype html
html(lang="en")
  head
    meta(charset="UTF-8")
    meta(http-equiv="X-UA-Compatible", content="IE=edge")
    meta(name="viewport", content="width=device-width, initial-scale=1.0")
    title Joom
    //- MVP CSS 사용
    link(rel="stylesheet", href="https://unpkg.com/mvp.css@1.12/mvp.css") 
  body 
    header
      h1 Joom 
    main 
      div#myStream
        video#myFace(autoplay, playsinline, width='400', height='400')
        button#mute Mute
        button#camera Turn Camera Off
        select#cameras 
    script(src="/socket.io/socket.io.js")
    script(src="/public/js/app.js")

```